In [21]:
# Установка зависимостей (один раз)
%pip install -q openai python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas

     ---------------------------------------- 11.3/11.3 MB 8.0 MB/s eta 0:00:00
     ------------------------------------- 347.8/347.8 kB 10.9 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
     ---------------------------------------- 12.9/12.9 MB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

In [6]:
import pandas as pd

# Укажите путь к файлу (на Windows удобно использовать raw-строку r"...")
csv_path = r"C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets.csv"  # например: r"C:\Users\USER\Downloads\data.csv"

df = pd.read_csv(csv_path, sep=';')  # при необходимости: sep=';', encoding='utf-8-sig' или 'cp1251'
df.head()

,Дата,Тип,Ссылка,Сообщение
0,01.08.2025 15:12,Пост,https://www.facebook.com/5465738283491903/post...,"🌍 Dans 12 pays d'Afrique, 80 000 enfants sont ..."
1,01.08.2025 10:04,Пост,https://telegram.me/conscientisation_panafrica...,"Au Ghana, le stock de la dette publique est pa..."
2,31.07.2025 19:59,Пост,https://www.tiktok.com/@actu_lome/video/753329...,Coupures de courant à Lomé : la CEET programme...
3,29.07.2025 18:56,Репост,https://www.facebook.com/25013882544880370,Il est des séquences politiques qui échappent ...
4,28.07.2025 18:35,Пост,https://telegram.me/conscientisation_panafrica...,𝗧𝗢𝗚𝗢 : 𝗟’𝗘𝗡𝗗𝗘𝗧𝗧𝗘𝗠𝗘𝗡𝗧 𝗤𝗨𝗜 𝗛𝗬𝗣𝗢𝗧𝗛𝗘̀𝗤𝗨𝗘 𝗟’𝗔𝗩𝗘𝗡𝗜𝗥 ...


In [ ]:
%pip install -q openai python-dotenv tqdm


In [1]:
import os
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd

# загружаем переменные окружения, где хранится OPENAI_API_KEY
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "OPENAI_API_KEY не найден в .env"



In [2]:
# список проблем на французском
FRENCH_PROBLEMS = [
    "Chômage", "Secteur informel", "Emploi précaire", "Inflation", "Pauvreté", "Inégalités",
    "Sous-alimentation", "Malnutrition", "Accès à l'eau", "Assainissement", "Logement insalubre",
    "Urbanisation", "Pollution", "Déforestation", "Changement climatique", "Sécheresse", "Inondations",
    "Corruption", "Mauvaise gouvernance", "Instabilité politique", "Insécurité", "Conflits",
    "Déplacement forcé", "Travail des enfants", "Mariage précoce", "Violence domestique", "Discrimination",
    "Accès à l'éducation", "Analphabétisme", "Sous-emploi des jeunes", "Système de santé faible",
    "Mortalité infantile", "Épidémies", "Coût des soins", "Manque de médicaments", "Infrastructure dégradée",
    "Transport limité", "Manque d'électricité", "Pénurie d'énergie", "Accès limité à internet",
    "Fuite des cerveaux", "Faible investissement", "Dépendance économique", "Dette publique",
    "Exportations limitées", "Fluctuation des prix", "Marché instable", "Tourisme faible",
    "Exploitation minière", "Agriculture peu productive"
]



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = (
    "Tu es un classificateur.: Donne exactement un seul label du jeu fourni (en français) qui est le plus pertinent et le plus fréquent dans le texte donné. "
    "Si aucune option n'est pertinente, réponds exactement par: 'нет'. Réponds uniquement par le label ou 'нет'."
)

USER_TEMPLATE = (
    "Liste de labels autorisés: {labels}\n\n"
    "Texte: \n{texte}\n\n"
    "Rends UNIQUEMENT le label le plus fréquent/parlant parmi la liste OU 'нет' si aucun ne s'applique."
)

def classify_text_to_label(text: str, labels: list[str]) -> str:
    if not isinstance(text, str) or not text.strip():
        return "нет"
    user_prompt = USER_TEMPLATE.format(labels=labels, texte=text[:2000])
    completion = client.chat.completions.create(
        model="gpt-4.1-nano", #gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=10,
    )
    answer = completion.choices[0].message.content.strip().strip(" .,:;\"'`")
    if answer.lower() == "нет":
        return "нет"
    normalized = {lbl.lower(): lbl for lbl in labels}
    key = answer.lower()
    return normalized.get(key, "нет")



In [20]:
# читаем исходный CSV с колонкой "сообщение"
# замените путь при необходимости
csv_path = r"C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets.csv"
df_full = pd.read_csv(csv_path, sep=';')
# если нужно, то можно сделать выборку
# df = df_full.sample(frac=0.01)
df = df_full
print(df.shape)

# применяем классификацию
labels = FRENCH_PROBLEMS
preds = []
for text in tqdm(df["Сообщение"].fillna("").astype(str), desc="Classifying"):
    preds.append(classify_text_to_label(text, labels))

df["проблема"] = preds

# сохраняем результат
out_path = csv_path.replace('.csv', '_classified.csv')
df.to_csv(out_path, index=False, sep=';')
out_path


(534, 4)


Classifying: 100%|██████████| 534/534 [03:45<00:00,  2.36it/s]


'C:\\Users\\USER\\progects\\Africa_analyse\\problems_study\\Togo_nets_classified.csv'

In [20]:
res_table = df['проблема'].value_counts()
res_table = res_table[(res_table > 1) & (res_table.index != 'нет')]
res_table



проблема
Corruption                    32
Pauvreté                      32
Dette publique                22
Inondations                   14
Inflation                     10
Conflits                       8
Violence domestique            8
Mauvaise gouvernance           7
Chômage                        6
Inégalités                     6
Accès à l'eau                  5
Changement climatique          5
Insécurité                     5
Agriculture peu productive     4
Pollution                      4
Assainissement                 4
Travail des enfants            3
Infrastructure dégradée        3
Malnutrition                   3
Discrimination                 2
Name: count, dtype: int64

In [3]:
os.listdir(r"C:\Users\USER\progects\Africa_analyse\problems_study")

['.env', 'main.ipynb', 'Togo_nets.csv', 'Togo_nets_classified.csv', 'venv']

In [6]:
df = pd.read_csv(r"C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets_classified.csv", sep=';')
df

,Дата,Тип,Ссылка,Сообщение,проблема
0,01.08.2025 15:12,Пост,https://www.facebook.com/5465738283491903/post...,"🌍 Dans 12 pays d'Afrique, 80 000 enfants sont ...",Inondations
1,01.08.2025 10:04,Пост,https://telegram.me/conscientisation_panafrica...,"Au Ghana, le stock de la dette publique est pa...",Dette publique
2,31.07.2025 19:59,Пост,https://www.tiktok.com/@actu_lome/video/753329...,Coupures de courant à Lomé : la CEET programme...,Mauvaise gouvernance
3,29.07.2025 18:56,Репост,https://www.facebook.com/25013882544880370,Il est des séquences politiques qui échappent ...,Chômage
4,28.07.2025 18:35,Пост,https://telegram.me/conscientisation_panafrica...,𝗧𝗢𝗚𝗢 : 𝗟’𝗘𝗡𝗗𝗘𝗧𝗧𝗘𝗠𝗘𝗡𝗧 𝗤𝗨𝗜 𝗛𝗬𝗣𝗢𝗧𝗛𝗘̀𝗤𝗨𝗘 𝗟’𝗔𝗩𝗘𝗡𝗜𝗥 ...,Dette publique
...,...,...,...,...,...
529,14.08.2024 14:22,Пост,https://telegram.me/strategiescontinentales/6986,🔴 🇹🇬 🇳🇪 Togo : Coopération bilatérale - Faure ...,Instabilité politique
530,13.08.2024 14:34,Пост,NaN,NaN,нет
531,13.08.2024 13:47,Пост,NaN,NaN,нет
532,02.08.2024 23:56,Пост,https://telegram.me/togoemploi/8800,💼 Recrutement d’une société d’assurance ou de ...,нет


In [8]:
problems_dict = {
    "Chômage": "Безработица",
    "Secteur informel": "Неформальный сектор",
    "Emploi précaire": "Нестабильная занятость",
    "Inflation": "Инфляция",
    "Pauvreté": "Бедность",
    "Inégalités": "Неравенство",
    "Sous-alimentation": "Недоедание",
    "Malnutrition": "Недостаточное питание",
    "Accès à l'eau": "Доступ к воде",
    "Assainissement": "Санитария",
    "Logement insalubre": "Непригодное жильё",
    "Urbanisation": "Урбанизация",
    "Pollution": "Загрязнение",
    "Déforestation": "Обезлесение",
    "Changement climatique": "Изменение климата",
    "Sécheresse": "Засуха",
    "Inondations": "Наводнения",
    "Corruption": "Коррупция",
    "Mauvaise gouvernance": "Плохое управление",
    "Instabilité politique": "Политическая нестабильность",
    "Insécurité": "Небезопасность",
    "Conflits": "Конфликты",
    "Déplacement forcé": "Вынужденное переселение",
    "Travail des enfants": "Детский труд",
    "Mariage précoce": "Ранние браки",
    "Violence domestique": "Домашнее насилие",
    "Discrimination": "Дискриминация",
    "Accès à l'éducation": "Доступ к образованию",
    "Analphabétisme": "Безграмотность",
    "Sous-emploi des jeunes": "Недоиспользование труда молодёжи",
    "Système de santé faible": "Слабая система здравоохранения",
    "Mortalité infantile": "Детская смертность",
    "Épidémies": "Эпидемии",
    "Coût des soins": "Стоимость медицинских услуг",
    "Manque de médicaments": "Недостаток лекарств",
    "Infrastructure dégradée": "Разрушенная инфраструктура",
    "Transport limité": "Ограниченный транспорт",
    "Manque d'électricité": "Отсутствие электричества",
    "Pénurie d'énergie": "Дефицит энергии",
    "Accès limité à internet": "Ограниченный доступ к интернету",
    "Fuite des cerveaux": "Утечка мозгов",
    "Faible investissement": "Низкие инвестиции",
    "Dépendance économique": "Экономическая зависимость",
    "Dette publique": "Государственный долг",
    "Exportations limitées": "Ограниченный экспорт",
    "Fluctuation des prix": "Колебания цен",
    "Marché instable": "Нестабильный рынок",
    "Tourisme faible": "Слабый туризм",
    "Exploitation minière": "Добыча полезных ископаемых",
    "Agriculture peu productive": "Малопродуктивное сельское хозяйство"
}


In [10]:
df['problem_russ'] = df['проблема'].map(problems_dict)
df

,Дата,Тип,Ссылка,Сообщение,проблема,problem_russ
0,01.08.2025 15:12,Пост,https://www.facebook.com/5465738283491903/post...,"🌍 Dans 12 pays d'Afrique, 80 000 enfants sont ...",Inondations,Наводнения
1,01.08.2025 10:04,Пост,https://telegram.me/conscientisation_panafrica...,"Au Ghana, le stock de la dette publique est pa...",Dette publique,Государственный долг
2,31.07.2025 19:59,Пост,https://www.tiktok.com/@actu_lome/video/753329...,Coupures de courant à Lomé : la CEET programme...,Mauvaise gouvernance,Плохое управление
3,29.07.2025 18:56,Репост,https://www.facebook.com/25013882544880370,Il est des séquences politiques qui échappent ...,Chômage,Безработица
4,28.07.2025 18:35,Пост,https://telegram.me/conscientisation_panafrica...,𝗧𝗢𝗚𝗢 : 𝗟’𝗘𝗡𝗗𝗘𝗧𝗧𝗘𝗠𝗘𝗡𝗧 𝗤𝗨𝗜 𝗛𝗬𝗣𝗢𝗧𝗛𝗘̀𝗤𝗨𝗘 𝗟’𝗔𝗩𝗘𝗡𝗜𝗥 ...,Dette publique,Государственный долг
...,...,...,...,...,...,...
529,14.08.2024 14:22,Пост,https://telegram.me/strategiescontinentales/6986,🔴 🇹🇬 🇳🇪 Togo : Coopération bilatérale - Faure ...,Instabilité politique,Политическая нестабильность
530,13.08.2024 14:34,Пост,NaN,NaN,нет,NaN
531,13.08.2024 13:47,Пост,NaN,NaN,нет,NaN
532,02.08.2024 23:56,Пост,https://telegram.me/togoemploi/8800,💼 Recrutement d’une société d’assurance ou de ...,нет,NaN


In [29]:
df['problem_russ'].value_counts()

problem_russ
Бедность                               32
Коррупция                              32
Государственный долг                   22
Наводнения                             14
Инфляция                               10
Конфликты                               8
Домашнее насилие                        8
Плохое управление                       7
Безработица                             6
Неравенство                             6
Доступ к воде                           5
Небезопасность                          5
Изменение климата                       5
Малопродуктивное сельское хозяйство     4
Загрязнение                             4
Санитария                               4
Детский труд                            3
Разрушенная инфраструктура              3
Недостаточное питание                   3
Дискриминация                           2
Name: count, dtype: int64

In [ ]:
# отфильтруем датафрейм, оставив только проблемы, которые встречаются более 1 раза
df_filt = df[df['проблема'].isin(res_table.index)]
df_filt.shape

(183, 6)

In [ ]:
# сохраняем отфильтрованный датафрейм
df_filt.to_csv(r'C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets_classified_filt.csv', index=False, sep=';')
final_problem_tables = df_filt['problem_russ'].value_counts()
final_problem_tables.to_csv(r'C:\Users\USER\progects\Africa_analyse\problems_study\Togo_nets_problem_table.csv', index=False, sep=';')

In [27]:
df = df_filt

In [28]:
import os
from datetime import datetime
import pandas as pd
from openai import OpenAI
from docx import Document

# =============================
# Конфигурация
# =============================
MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1-mini")
MAX_MESSAGES_PER_PROBLEM = 7
MAX_CHARS_PER_MESSAGE = 1500
OUTPUT_DOCX_PATH = f"togo_problems_report_{datetime.now().strftime('%Y%m%d_%H%M')}.docx"

# =============================
# Проверки окружения и колонок
# =============================
if "df" not in globals():
    raise RuntimeError(
        "В памяти должен быть датафрейм 'df'. Убедитесь, что вы его подготовили в предыдущих ячейках."
    )

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "Не обнаружен OPENAI_API_KEY. Установите переменную окружения OPENAI_API_KEY с вашим ключом."
    )

# Определяем имена колонок
problem_col = None
for col in ["проблема", "problem", "Проблема"]:
    if col in df.columns:
        problem_col = col
        break
if problem_col is None:
    raise ValueError("Не найдена колонка с типом проблемы ('проблема'/'problem').")

problem_ru_col = None
for col in ["problem_russ", "problem_ru", "проблема_ru", "проблема_rus", "проблема_рус", "проблема_перевод"]:
    if col in df.columns:
        problem_ru_col = col
        break

message_col = None
for col in ["Сообщение", "Сообщения", "сообщение", "text", "Текст"]:
    if col in df.columns:
        message_col = col
        break
if message_col is None:
    raise ValueError("Не найдена колонка с текстами сообщений ('Сообщение'/'Сообщения').")

link_col = "Ссылка" if "Ссылка" in df.columns else None

# =============================
# Вспомогательные функции
# =============================
client = OpenAI()

def normalize_message(msg: object) -> str | None:
    if pd.isna(msg):
        return None
    s = str(msg).strip()
    if len(s) > MAX_CHARS_PER_MESSAGE:
        s = s[:MAX_CHARS_PER_MESSAGE] + "..."
    return s if s else None


def build_prompt(problem_key: str, problem_ru: str | None, messages: list[str]):
    msgs_block = "\n\n".join([f"- {m}" for m in messages])
    problem_title = f"{problem_key}" + (f" / {problem_ru}" if problem_ru else "")

    sys = (
        "Ты профессиональный социолог. "
        "Проанализируй высказывания жителей государства Того в соцсетях и опиши проблемы, которые они поднимают. "
        "Пиши на русском языке, чётко и по делу."
    )

    usr = f"""
Тип проблемы: {problem_title}

Высказывания (до {MAX_MESSAGES_PER_PROBLEM} шт):
{msgs_block}

Задача:
1) Сформулируй суть ключевых проблем, описанных в текстах
2) Подробно опиши их проявления. Выбери наиболее интересные и яркие примеры. ОБЯЗАТЕЛЬНО ВКЛЮЧИ ИХ В ОТЧЕТ, если они описаны в текстах!!!
3) Укажи, кого это затрагивает сильнее всего (если понятно).

Важно:
- ОБЯЗАТЕЛЬНО включи несколько цитат из списка выше, которые иллюстрируют выводы, и сразу дай им перевод на русский в скобках.
- Структурируй текст подзаголовками.
- Если в тексте есть персональные данные, то обязательно используй их в отчете.
"""
    return [
        {"role": "system", "content": sys},
        {"role": "user", "content": usr.strip()},
    ]


def ask_gpt(messages):
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.2,
        max_tokens=900,
    )
    return resp.choices[0].message.content.strip()


def collect_links(df_subset: pd.DataFrame, max_links: int = 3) -> list[str]:
    if link_col is None:
        return []
    links = (
        df_subset[link_col]
        .dropna()
        .astype(str)
        .drop_duplicates()
        .head(max_links)
        .tolist()
    )
    return links


def add_section(doc: Document, problem_key: str, problem_ru: str | None, analysis_text: str, links: list[str]):
    heading = f"{problem_key}" + (f" — {problem_ru}" if problem_ru else "")
    doc.add_heading(heading, level=1)
    for paragraph in analysis_text.split("\n\n"):
        doc.add_paragraph(paragraph)
    if links:
        doc.add_paragraph("Источники и материалы:")
        for url in links:
            doc.add_paragraph(url)

# =============================
# Формирование отчёта
# =============================
problem_counts = df[problem_col].value_counts(dropna=True)
problems_ordered = list(problem_counts.index) #["Corruption"] 

document = Document()
document.add_heading("Проблемы жителей Того: тематический анализ", 0)
document.add_paragraph(
    f"Дата формирования отчёта: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
)

for problem in problems_ordered:
    subset = df[df[problem_col] == problem].copy()
    subset = subset[~subset[message_col].isna()]
    if subset.empty:
        continue

    messages = (
        subset[message_col]
        .map(normalize_message)
        .dropna()
        .tolist()[:MAX_MESSAGES_PER_PROBLEM]
    )
    if not messages:
        continue

    problem_ru = (
        subset[problem_ru_col].iloc[0]
        if problem_ru_col and problem_ru_col in subset.columns
        else None
    )

    prompt_msgs = build_prompt(problem, problem_ru, messages)
    print(f"Анализ: {problem} ({len(messages)} сообщений)...")

    try:
        analysis = ask_gpt(prompt_msgs)
    except Exception as e:
        analysis = f"Не удалось получить анализ из модели. Ошибка: {e}"

    links = collect_links(subset, max_links=3)
    add_section(document, problem, problem_ru, analysis, links)

# Сохранение отчёта
document.save(OUTPUT_DOCX_PATH)
print(f"Отчёт сохранён: {OUTPUT_DOCX_PATH}")


Анализ: Pauvreté (7 сообщений)...
Анализ: Corruption (7 сообщений)...
Анализ: Dette publique (7 сообщений)...
Анализ: Inondations (7 сообщений)...
Анализ: Inflation (7 сообщений)...
Анализ: Conflits (7 сообщений)...
Анализ: Violence domestique (7 сообщений)...
Анализ: Mauvaise gouvernance (7 сообщений)...
Анализ: Chômage (6 сообщений)...
Анализ: Inégalités (6 сообщений)...
Анализ: Accès à l'eau (5 сообщений)...
Анализ: Insécurité (5 сообщений)...
Анализ: Changement climatique (5 сообщений)...
Анализ: Agriculture peu productive (4 сообщений)...
Анализ: Pollution (4 сообщений)...
Анализ: Assainissement (4 сообщений)...
Анализ: Travail des enfants (3 сообщений)...
Анализ: Infrastructure dégradée (3 сообщений)...
Анализ: Malnutrition (3 сообщений)...
Анализ: Discrimination (2 сообщений)...
Отчёт сохранён: togo_problems_report_20250814_1222.docx
